In [ ]:
import streamlit as st
import pickle
import pandas as pd

# Cargar el modelo
with open("../03_EDA/stacking_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

# Cargar los LabelEncoders y la lista de características
with open("../03_EDA/preprocessing.pkl", "rb") as pre_file:
    preprocessing = pickle.load(pre_file)

encoder_exclusivity = preprocessing["encoder_exclusivity"]
encoder_sizecategory = preprocessing["encoder_sizecategory"]
features = preprocessing["features"]

# Cargar el ranking de inversión
df_ranking = pd.read_csv("../03_EDA/ranking_lego_web.csv")

# Streamlit App
st.title("Plataforma de Recomendación de LEGO 📊")

# Mostrar el ranking de inversión
st.subheader("Ranking de Inversión LEGO")
st.dataframe(df_ranking)

# Entrada del usuario para predecir inversión en un set nuevo
st.subheader("Predicción para un Nuevo Set")
us_retail_price = st.number_input("Precio en USD", min_value=1.0, max_value=1000.0)
pieces = st.number_input("Número de Piezas", min_value=1, max_value=10000)
minifigs = st.number_input("Número de Minifigs", min_value=0, max_value=50)
years_since_exit = st.number_input("Años desde el retiro", min_value=0, max_value=50)
resale_demand = st.number_input("Demanda de reventa", min_value=0.0, max_value=10000.0)
annual_price_increase = st.number_input("Incremento Anual del Precio", min_value=-100.0, max_value=1000.0)

exclusivity = st.selectbox("Exclusividad", encoder_exclusivity.classes_)
size_category = st.selectbox("Tamaño del Set", encoder_sizecategory.classes_)

# Convertir categorías en valores numéricos
exclusivity_encoded = encoder_exclusivity.transform([exclusivity])[0]
size_category_encoded = encoder_sizecategory.transform([size_category])[0]

# Crear el dataframe de entrada
df_input = pd.DataFrame([[
    us_retail_price, pieces, minifigs, years_since_exit, resale_demand, annual_price_increase,
    exclusivity_encoded, size_category_encoded, us_retail_price / pieces, 
    us_retail_price / max(minifigs, 1), years_since_exit
]], columns=features)

# Predecir el Investment Score
if st.button("Predecir Potencial de Inversión"):
    predicted_score = model.predict(df_input)[0]
    st.success(f"Predicted Investment Score: {predicted_score:.2f}")
